# Regional Salary Data Extraction Tool

## Purpose
This notebook extracts and compiles salary data from multiple Excel files into a single consolidated dataset.

## Input Requirements
- Excel files must have two sheets:
  - 'Input Data' sheet with headers in A4:A5 and values in B4:B5
  - 'Salaries' sheet with headers in A3:F3 and data starting from row 4

## Output
- Creates 'compiled_data.xlsx' containing consolidated salary data from all input files

In [30]:
import os
import pandas as pd

In [31]:
def process_spreadsheet(file_path):
    """
    Process an individual Excel spreadsheet and extract salary data.
    
    Args:
        file_path (str): Path to the Excel file
        
    Returns:
        pandas.DataFrame: Processed data from the spreadsheet
        None: If there's an error processing the file
    """
    try:
        # Read Input Data sheet
        input_data = pd.read_excel(file_path, sheet_name='Input Data')
        
        # Fixed indices to match actual data location
        header1 = input_data.iloc[2, 0]  # A3
        header2 = input_data.iloc[3, 0]  # A4
        value1 = input_data.iloc[2, 1]   # B3
        value2 = input_data.iloc[3, 1]   # B4
        
        # Read Salaries sheet, skipping two rows to get to the header row
        salary_data = pd.read_excel(file_path, sheet_name='Salaries', header=2)
        
        # Remove any completely empty rows
        salary_data = salary_data.dropna(how='all')
        
        # Add the input data columns
        salary_data[header1] = value1
        salary_data[header2] = value2
        
        # Add source file column
        salary_data['Source File'] = os.path.basename(file_path)
        
        return salary_data
        
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return None

In [32]:
def main():
    folder_path = r"C:\Users\scott\OneDrive\Desktop\DS4B_301P_FASTRACK\excel_extract_salaries\data"
    all_data = []
    first_file_columns = None
    
    print(f"Looking for Excel files in: {folder_path}")
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            try:
                df = process_spreadsheet(file_path)
                if df is not None:
                    if first_file_columns is None:
                        first_file_columns = df.columns.tolist()
                    else:
                        # Ensure all dataframes have the same columns as the first file
                        for col in first_file_columns:
                            if col not in df.columns:
                                df[col] = ''  # Add missing columns with empty values
                        df = df[first_file_columns]  # Reorder columns to match first file
                    all_data.append(df)
                    print(f"Successfully processed: {filename}")
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
    
    if not all_data:
        print("No data was processed successfully. Please check the files and permissions.")
        return

    # Combine all DataFrames
    final_df = pd.concat(all_data, ignore_index=True)
    
    # Export to Excel in output directory
    output_dir = r"C:\Users\scott\OneDrive\Desktop\DS4B_301P_FASTRACK\excel_extract_salaries\data"
    os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist
    output_path = os.path.join(output_dir, 'compiled_data.xlsx')
    
    try:
        final_df.to_excel(output_path, index=False)
        print(f"\nData compiled and exported to {output_path}")
        print(f"Total rows in output: {len(final_df)}")
    except Exception as e:
        print(f"Error exporting data: {str(e)}")

In [33]:
# Run the script
if __name__ == "__main__":
    main()

Looking for Excel files in: C:\Users\scott\OneDrive\Desktop\DS4B_301P_FASTRACK\excel_extract_salaries\data
Successfully processed: Salary_Report_BatonRouge.xlsx
Successfully processed: Salary_Report_Lafayette.xlsx
Successfully processed: Salary_Report_Monroe.xlsx
Successfully processed: Salary_Report_NewOrleans.xlsx
Successfully processed: Salary_Report_Northeast.xlsx
Successfully processed: Salary_Report_Plaquemines.xlsx

Data compiled and exported to C:\Users\scott\OneDrive\Desktop\DS4B_301P_FASTRACK\excel_extract_salaries\data\compiled_data.xlsx
Total rows in output: 24
